In [15]:
%%writefile assignment42.cu
// Записываем код  в файл для компиляции CUDA
// Assignment 4 Task 2
// Вычислить префиксной суммы (сканирования) массива с использованием разделяемой памяти
// Сравните время выполнения с последовательной реализацией на CPU для массива размером 1 000 000 элементов.

#include <iostream>               // Для стандартный ввод-вывод
#include <cuda_runtime.h>         // Для CUDA Runtime API
#include <chrono>                 // Для измерения времени

using namespace std;              // Чтобы не писать std::

// CUDA KERNEL
__global__ void sumKernel(int *d_array, int *d_result, int n) {                 // CUDA-ядро для суммирования элементов массива
    int idx = threadIdx.x + blockIdx.x * blockDim.x;                            // Вычисляем глобальный индекс потока (каждый поток работает со своим элементом массива)

    if (idx < n)                                                                // Проверяем, чтобы индекс не выходил за пределы массива
        atomicAdd(d_result, d_array[idx]);                                      // Атомарно добавляем значение элемента к результату (защищает от гонок данных)
}

// Основная функция
int main() {
    const int N = 1000000;                                                      // размер массива
    const int SIZE = N * sizeof(int);                                           // размер массива в байтах для выделения памяти

    int *h_array = new int[N];                                                  // Выделяем память на CPU для исходного массива
    int h_result_cpu = 0;                                                       // Переменная для хранения суммы на CPU
    int h_result_gpu = 0;                                                       // Переменная для хранения суммы на GPU

    // Инициализация массива
    for (int i = 0; i < N; i++)                                                 // Проходим по всем элементам массива
        h_array[i] = 1;                                                         // Заполняем массив единицами (для простоты проверки)

    // CPU сумма
    auto start_cpu = chrono::high_resolution_clock::now();                      // Фиксируем старт таймера для CPU
    for (int i = 0; i < N; i++)                                                 // Проходим по массиву последовательно
        h_result_cpu += h_array[i];                                             // Складываем элементы массива
    auto end_cpu = chrono::high_resolution_clock::now();                        // Фиксируем окончание работы CPU
    chrono::duration<double> cpu_time = end_cpu - start_cpu;                    // Вычисляем продолжительность работы CPU в секундах

    // GPU сумма
    int *d_array, *d_result;                                                    // Указатели для массива и результата на GPU
    cudaMalloc(&d_array, SIZE);                                                 // Выделяем память на GPU под исходный массив
    cudaMalloc(&d_result, sizeof(int));                                         // Выделяем память на GPU под результат суммы
    cudaMemcpy(d_array, h_array, SIZE, cudaMemcpyHostToDevice);                 // Копируем данные с CPU на GPU
    cudaMemset(d_result, 0, sizeof(int));                                       // Обнуляем результат на GPU перед запуском ядра

    int threadsPerBlock = 256;                                                  // Количество потоков в одном блоке
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;            // Вычисляем количество блоков, чтобы покрыть весь массив

    cudaEvent_t start_gpu, stop_gpu;                                            // Создаём события CUDA для измерения времени GPU
    cudaEventCreate(&start_gpu);                                                // Инициализация события старта
    cudaEventCreate(&stop_gpu);                                                 // Инициализация события конца
    cudaEventRecord(start_gpu);                                                 // Запускаем событие старта

    sumKernel<<<blocksPerGrid, threadsPerBlock>>>(d_array, d_result, N);        // Запуск ядра CUDA

    cudaEventRecord(stop_gpu);                                                  // Отмечаем событие конца
    cudaEventSynchronize(stop_gpu);                                             // Ждём завершения ядра GPU
    float gpu_time = 0;                                                         // Переменная для хранения времени GPU
    cudaEventElapsedTime(&gpu_time, start_gpu, stop_gpu);                       // Вычисляем время выполнения ядра в миллисекундах

    cudaMemcpy(&h_result_gpu, d_result, sizeof(int), cudaMemcpyDeviceToHost);   // Копируем результат с GPU на CPU

    // Вывод результатов
    cout << "Результат CPU: " << h_result_cpu << endl;                          // Выводим результат работы CPU
    cout << "Результат GPU: " << h_result_gpu << endl;                          // Выводим результат работы GPU
    cout << "Время CPU: " << cpu_time.count() * 1000 << " мс" << endl;          // Выводим время CPU в миллисекундах
    cout << "Время GPU: " << gpu_time << " мс" << endl;                         // Выводим время GPU в миллисекундах

    // Вычисляем ускорение
    double speedup = (cpu_time.count() * 1000) / gpu_time;                      // Вычисляем на сколько раз GPU быстрее CPU
    cout << "GPU работает примерно в " << speedup << " раз быстрее CPU." << endl; // Выводим ускорение

    // Освобождение памяти
    delete[] h_array;                                       // Освобождаем память CPU
    cudaFree(d_array);                                      // Освобождаем память GPU для массива
    cudaFree(d_result);                                     // Освобождаем память GPU для результата

    return 0;                                               // Завершаем программу
}


Overwriting assignment42.cu


In [16]:
# Компиляция
!nvcc assignment42.cu -o assignment42 -arch=sm_75 -std=c++11            # -arch=sm_75  - архитектура GPU (Tesla T4 в Colab = sm_75)
                                                                        # -std=c++11 — стандарт C++
# Запуск
!./assignment42

Результат CPU: 1000000
Результат GPU: 1000000
Время CPU: 2.27392 мс
Время GPU: 0.104 мс
GPU работает примерно в 21.8646 раз быстрее CPU.
